In [1]:
import pandas as pd

In [2]:
import os

In [3]:
from pyspark.sql import SparkSession

In [4]:
import plotly.express as px

In [5]:
import numpy as np

In [6]:
spark = SparkSession\
        .builder\
        .master("local")\
        .appName('flights')\
        .getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
# Change this list to include/exclude columns in the df
cols = {'Year':str,
         'Quarter':float,
         'Month':float,
         'DayofMonth':float,
         'DayOfWeek':float,
         'Reporting_Airline':str,
         'Origin':str,
         'Dest':str,
         'DepTime':float,
         'DepDelay':float,
         'DepDelayMinutes':float,
         'DepDel15':float,
         'TaxiOut':float,
         'TaxiIn':float,
         'ArrTime':float,
         'ArrDelay':float,
         'ArrDelayMinutes':float,
         'ArrDel15':float,
         'Cancelled':float,
         'CarrierDelay':float,
         'WeatherDelay':float,
         'NASDelay':float,
         'SecurityDelay':float,
         'LateAircraftDelay':float,
         'DivArrDelay':float}

In [9]:
months = ['January',
          'February',
          'March',
          'April',
          'May',
          'June',
          'July',
          'August',
          'September',
          'October',
          'November',
          'December']

**Populate Data**

In [10]:
c = 0
for yr in ['2005','2006','2007','2013','2014','2015']:
    for file, month in zip(os.listdir('./data/' + yr + '/CSV/'), months):
        df = pd.read_csv('./data/' + yr + '/CSV/' + str(file))
        miaDf = df.loc[((df['Origin']=='MIA')|(df['Dest']=='MIA')),list(cols.keys())].copy()
        if c == 0:
            miaSpark = spark.createDataFrame(miaDf)
            c += 1
        else:
            miaSpark = miaSpark.union(spark.createDataFrame(miaDf))
        print('Populated ' + month + ' ' + yr)
        del(miaDf)
        del(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.



Populated January 2005
Populated February 2005
Populated March 2005
Populated April 2005
Populated May 2005
Populated June 2005
Populated July 2005
Populated August 2005


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (37,48) have mixed types.Specify dtype option on import or set low_memory=False.



Populated September 2005
Populated October 2005
Populated November 2005


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.



...

Populated January 2014
Populated February 2014
Populated March 2014
Populated April 2014
Populated May 2014
Populated June 2014
Populated July 2014
Populated August 2014


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (48,77,84) have mixed types.Specify dtype option on import or set low_memory=False.



Populated September 2014
Populated October 2014
Populated November 2014
Populated December 2014
Populated January 2015
Populated February 2015
Populated March 2015
Populated April 2015
Populated May 2015
Populated June 2015
Populated July 2015
Populated August 2015


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (76,77,84,85) have mixed types.Specify dtype option on import or set low_memory=False.



Populated September 2015
Populated October 2015
Populated November 2015
Populated December 2015


## Departure Delays Origin = MIA

In [154]:
origDelays=\
miaSpark.select('Year', 'Month','TaxiOut', 'DepDelayMinutes', 'ArrDelayMinutes')\
        .dropna(how='any')\
        .filter("Origin == 'MIA' and (Year == 2005 or Year == 2015)")\
        .groupBy('Year', "Month")\
        .mean('TaxiOut', 'DepDelayMinutes','ArrDelayMinutes')\
        .sort("Year","Month")\
        .toPandas()

In [155]:
origDelays.Year = origDelays.Year.astype(str)

In [156]:
origDelays.Month = origDelays.Month.replace({1:"January",
                                           2:"February",
                                           3:"March",
                                           4:"April",
                                           5:"May",
                                           6:"June",
                                           7:"July",
                                           8:"August",
                                           9:"September",
                                          10:"October",
                                          11:"November",
                                          12:"December"})

In [157]:
fig = px.line(origDelays,
              x='Month', 
              y='avg(DepDelayMinutes)',
              color='Year',
              labels={'avg(DepDelayMinutes)':"Average Departure Delay (minutes)"},
              title="Average Departure Delays From MIA Before And After Renovation")

In [158]:
fig.show()

## Arrival Delays Origin = MIA

In [159]:
fig = px.line(origDelays,
              x='Month', 
              y='avg(ArrDelayMinutes)',
              color='Year',
              labels={'avg(ArrDelayMinutes)':"Average Arrival Delay (minutes)"},
              title="Average Arrival Delays From MIA Before And After Renovation")

In [160]:
fig.show()

## Taxi Out Time

In [161]:
fig = px.line(origDelays,
              x='Month', 
              y='avg(TaxiOut)',
              color='Year',
              labels={'avg(TaxiOut)':"Average Taxi Out Time (minutes)"},
              title="Average Taxi Out Time At MIA Before And After Renovation")

In [162]:
fig.show()

## Taxi In Time

In [163]:
destDelays=\
miaSpark.select('Year', 'Month','TaxiIn')\
        .dropna(how='any')\
        .filter("Dest == 'MIA' and (Year == 2005 or Year == 2015)")\
        .groupBy('Year', "Month")\
        .mean('TaxiIn')\
        .sort("Year","Month")\
        .toPandas()

In [164]:
destDelays.Year = destDelays.Year.astype(str)

In [165]:
destDelays.Month = destDelays.Month.replace({1:"January",
                                           2:"February",
                                           3:"March",
                                           4:"April",
                                           5:"May",
                                           6:"June",
                                           7:"July",
                                           8:"August",
                                           9:"September",
                                          10:"October",
                                          11:"November",
                                          12:"December"})

In [166]:
fig = px.line(destDelays,
              x='Month', 
              y='avg(TaxiIn)',
              color='Year',
              labels={'avg(TaxiIn)':"Average Taxi In Time (minutes)"},
              title="Average Taxi In Time At MIA Before And After Renovation")

In [167]:
fig.show()

## Security Delays

In [168]:
secDelay = \
miaSpark.filter("securityDelay>0 and Origin == 'MIA' and (Year == 2005 or Year == 2015)")\
        .select("Year","Month","securityDelay")\
        .dropna(how='any')\
        .groupBy("Year","Month")\
        .mean("securityDelay")\
        .sort("Year","Month")\
        .toPandas()

In [169]:
secDelay.Year = secDelay.Year.astype(str)

In [170]:
secDelay.Month = secDelay.Month.replace({1:"January",
                                           2:"February",
                                           3:"March",
                                           4:"April",
                                           5:"May",
                                           6:"June",
                                           7:"July",
                                           8:"August",
                                           9:"September",
                                          10:"October",
                                          11:"November",
                                          12:"December"})

In [171]:
fig = px.line(secDelay,x='Month', 
              y='avg(securityDelay)',
              color='Year',
              labels={'avg(securityDelay)':"Average Security Delay (minutes)"},
              title="Average Security Delays At MIA Before and After Renovation")

In [172]:
fig.show()

## Traffic

In [173]:
traffic=\
miaSpark.filter("(Origin='MIA' or Dest='MIA') and (Year == 2005 or Year == 2015)")\
        .select("Year","Month","Reporting_Airline")\
        .groupBy("Year","Month")\
        .count()\
        .sort("Year","Month")\
        .toPandas()

In [174]:
traffic.Year = traffic.Year.astype(str)

In [175]:
traffic.Month = traffic.Month.replace({1:"January",
                                           2:"February",
                                           3:"March",
                                           4:"April",
                                           5:"May",
                                           6:"June",
                                           7:"July",
                                           8:"August",
                                           9:"September",
                                          10:"October",
                                          11:"November",
                                          12:"December"})

In [176]:
fig = px.bar(traffic, 
             x='Month',
             y="count",
             color="Year", 
             barmode='group',
             labels={'counts':"Flights"},
             title="Total Number of Flights At MIA Before And After Renovation")

fig.show()

## Adjusting For Increase Traffic

In [177]:
trafficAdj = traffic.groupby('Year').sum('count').reset_index()

In [178]:
trafficAdj['count'] = trafficAdj['count'].apply(lambda x: (1-(x/min(trafficAdj['count'])-1)))

In [179]:
trafficAdj.set_index("Year", inplace=True)

In [180]:
origDelaysAdj = origDelays.set_index("Year").join(trafficAdj)

In [181]:
origDelaysAdj['adjAvgDepDelay'] = origDelaysAdj['avg(DepDelayMinutes)'] * origDelaysAdj['count']

In [188]:
origDelaysAdj['adjAvgArrDelay'] = origDelaysAdj['avg(ArrDelayMinutes)'] * origDelaysAdj['count']

In [182]:
origDelaysAdj['adjTaxiOut'] = origDelaysAdj['avg(TaxiOut)'] * origDelaysAdj['count']

In [183]:
origDelaysAdj.reset_index(inplace = True)

## Adjusted Departure Delays

In [189]:
fig = px.line(origDelaysAdj,
              x='Month', 
              y='adjAvgDepDelay',
              color='Year',
              labels={'adjAvgDepDelay':"Average Departure Delay (minutes)"},
              title='Average Departure Delays From MIA Adjusted For Increased Traffic')

In [190]:
fig.show()

## Adjusted Arrival Delays Origin = MIA

In [191]:
fig = px.line(origDelaysAdj,
              x='Month', 
              y='adjAvgArrDelay',
              color='Year',
              labels={'adjAvgArrDelay':"Average Arrival Delay (minutes)"},
              title="Average Arrival Delays From MIA Adjusted For Increased Traffic")

In [192]:
fig.show()

## Adjusted Taxi Out Time

In [197]:
fig = px.line(origDelaysAdj,
              x='Month', 
              y='adjTaxiOut',
              color='Year',
              labels={'adjTaxiOut':"Average Taxi Out Time (minutes)"},
              title="Average Taxi Out Time At MIA Adjusted For Increased Traffic")

In [198]:
fig.show()

## Adjusted Taxi In Time

In [193]:
destDelaysAdj = destDelays.set_index("Year").join(trafficAdj)

In [194]:
destDelaysAdj['adjTaxiIn'] = destDelaysAdj['avg(TaxiIn)'] * destDelaysAdj['count']

In [196]:
destDelaysAdj.reset_index(inplace = True)

In [199]:
fig = px.line(destDelaysAdj,
              x='Month', 
              y='adjTaxiIn',
              color='Year',
              labels={'adjTaxiIn':"Average Taxi In Time (minutes)"},
              title="Average Taxi In Time At MIA Adjusted For Increased Traffic")

In [200]:
fig.show()

## Adjusted Security Delays

In [202]:
secDelaysAdj = secDelay.set_index("Year").join(trafficAdj)

In [203]:
secDelaysAdj['adjSecurityDelay'] = secDelaysAdj['avg(securityDelay)'] * secDelaysAdj['count']

In [204]:
secDelaysAdj.reset_index(inplace = True)

In [206]:
fig = px.line(secDelaysAdj,x='Month', 
              y='adjSecurityDelay',
              color='Year',
              labels={'adjSecurityDelay':"Average Security Delay (minutes)"},
              title="Average Security Delays At MIA Adjusted For Increased Traffic")

In [207]:
fig.show()